In [1]:
import deepchem
import xgboost
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter

import numpy as np
from sklearn.model_selection import GridSearchCV

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [9]:
rocAucMetric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.roc_auc_score)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
convMolFeat = deepchem.deepchem.feat.ConvMolFeaturizer()
xgb_reg_ecfp = xgboost.XGBClassifier()
xgb_reg_estate = xgboost.XGBClassifier()
praucMetric = deepchem.deepchem.metrics.prc_auc_score
praucMetricWrapped = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.prc_auc_score)

"""params = {'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}"""

params = {"max_depth": [3, 4]}

classificationDatasets = ['HIA_Hou', 'Pgp_Broccatelli', 'Bioavailability_Ma', 'BBB_Martins', 'CYP2C9_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith', 'CYP2C9_Substrate_CarbonMangels', 'CYP2D6_Substrate_CarbonMangels', 'CYP3A4_Substrate_CarbonMangels', 'hERG', 'AMES', 'DILI']

for dataset in ['CYP2D6_Substrate_CarbonMangels', 'HIA_Hou']:

    xgbScoring = None
    gcnScoring = None

    if(dataset in ['CYP2C9_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith', 'CYP2C9_Substrate_CarbonMangels','CYP2D6_Substrate_CarbonMangels']):
           xgbScoring = praucMetric
           gcnScoring = praucMetricWrapped
           print("prauc")
    else:
           xgbScoring = deepchem.deepchem.metrics.roc_auc_score
           gcnScoring = rocAucMetric
           print("rocauc")

    benchmark = group.get(dataset)
    name = benchmark["name"]
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = 1)  
    results = {}

    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost (estate)
    trainValEstateFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testEstateFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #featurize training, valid, and test data for xgboost (ecfp)
    trainValEcfpFeat = ecfpFeat.featurize(train_val.iloc[:,1].to_list())
    testEcfpFeat = ecfpFeat.featurize(test.iloc[:,1].to_list())

    #featurize training, valid, and test data for the GCN
    trainGcnFeat = convMolFeat.featurize(train.iloc[:,1].to_list())
    validGcnFeat = convMolFeat.featurize(valid.iloc[:,1].to_list())
    testGcnFeat = convMolFeat.featurize(test.iloc[:,1].to_list())

    #convert training and validation data into a deepchem dataset for the gcn
    gcn_train_data = deepchem.deepchem.data.NumpyDataset(X=trainGcnFeat, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    gcn_valid_data = deepchem.deepchem.data.NumpyDataset(X=validGcnFeat, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    #tune estate classifier
    clfEstate = GridSearchCV(estimator=xgb_reg_estate, param_grid=params, scoring=xgbScoring, cv=10)
    clfEstate.fit(trainValEstateFeat, train_val.iloc[:,2])
    results["estate"] = clfEstate.best_params_

    #tune ecfp classifier
    clfEcfp = GridSearchCV(estimator=xgb_reg_ecfp, param_grid=params, scoring=xgbScoring, cv=10)
    clfEcfp.fit(trainValEcfpFeat, train_val.iloc[:,2])
    results["ecfp"] = clfEcfp.best_params_

    def model_builder(**model_params):
       return deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="classification", graph_conv_layers=[128, 128, 64])

    optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

    params_dict = {"dropout": .01,} #"dense_layer_size": 200}

    best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, gcn_train_data, gcn_valid_data, gcnScoring, search_range=2)
    results["gcn"] = best_hyperparams

    print(name)
    print(results)

    predictions_list = []

    for seed in [1, 2, 3, 4, 5]:

      predictions = {}

      trainBench, validBench = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
      ecfp_benchmark_reg = xgboost.XGBClassifier(max_depth=results["ecfp"]["max_depth"])
      estate_benchmark_reg = xgboost.XGBClassifier(max_depth=results["estate"]["max_depth"])

      #colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=5000

      trainGcnFeatBench = convMolFeat.featurize(trainBench.iloc[:,1].to_list())
      validGcnFeatBench = convMolFeat.featurize(validBench.iloc[:,1].to_list())

      gcn_bench_train_data = deepchem.deepchem.data.NumpyDataset(X=trainGcnFeatBench, y=np.array(trainBench.iloc[:,2]), ids=np.array(trainBench.iloc[:,1].to_list()))
      gcn_bench_valid_data = deepchem.deepchem.data.NumpyDataset(X=validGcnFeatBench, y=np.array(validBench.iloc[:,2]), ids=np.array(validBench.iloc[:,1].to_list()))

      reg = deepchem.deepchem.models.GraphConvModel(
         n_tasks=1, 
         dropout=results["gcn"]["dropout"],
         dense_layer_size=400,#results["gcn"]["dense_layer_size"],
         graph_conv_layers=[128, 128, 64],
         mode="classification",)
      callback = deepchem.deepchem.models.ValidationCallback(gcn_bench_valid_data, 1000, gcnScoring)
      reg.fit(gcn_bench_train_data, nb_epoch=100, callbacks=callback)

      #I don't think eval_metric does anything if there's no eval_set specified,
      #but if there is one, I think you're supposed to pass a string like "mae"
      estate_benchmark_reg.fit(X=trainValEstateFeat, y=train_val.iloc[:,2], eval_metric=xgbScoring, verbose=True)
      ecfp_benchmark_reg.fit(X=trainValEcfpFeat, y=train_val.iloc[:,2], eval_metric=xgbScoring)
      
      pred_estate = estate_benchmark_reg.predict(X=testEstateFeat)
      pred_ecfp = ecfp_benchmark_reg.predict(X=testEcfpFeat)

      gcn_pred_raw = reg.predict_on_batch(X=np.array(testGcnFeat))
      gcn_pred = []
      for prediction in gcn_pred_raw:
       gcn_pred.append(prediction[0][1])

      print(gcn_pred[0:5])
      print(pred_ecfp[0:5])
      print(pred_estate[0:5])

      y_pred_test = np.round(np.mean([ pred_estate, pred_ecfp, gcn_pred ], axis=0))

      print("predictions for run #")
      print(seed)
      print(y_pred_test[0:5])

      predictions[name] = y_pred_test
      predictions_list.append(predictions)

    print(predictions_list)
    print(group.evaluate_many(predictions_list))



generating training, validation splits...


prauc


100%|██████████| 532/532 [00:00<00:00, 1989.69it/s]


[00:11:07] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:11:07] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:11:07] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:11:07] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation me

generating training, validation splits...


...

generating training, validation splits...


[1.0, 0.9999995, 0.9999989, 0.9999943, 0.9999932]
[1 1 1 1 1]
[1 1 1 1 1]
predictions for run #
4
[1. 1. 1. 1. 1.]


100%|██████████| 461/461 [00:00<00:00, 2516.71it/s]


[01:13:25] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:13:25] WARNING: D:\bld\xgboost-split_1643227225381\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.99999964, 0.9999949, 0.9999981, 0.9999949, 0.9998455]
[1 1 1 1 1]
[1 1 1 1 1]
predictions for run #
5
[1. 1. 1. 1. 1.]
[{'cyp2d6_substrate_carbonmangels': array([0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 

KeyError: 'cyp2d6_substrate_carbonmangels'

In [19]:
group.evaluate_many(predictions_list[5:11])

{'hia_hou': [0.818, 0.021]}